In [1]:
import os
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import PowerTransformer

from imblearn.over_sampling import SMOTE

## models
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
from sklearn.svm import LinearSVC

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, cross_val_predict, cross_validate

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix, make_scorer, roc_auc_score

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

## to make it possible to display multiple output inside one cell 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.options.display.max_rows = 50
pd.set_option('display.float_format', lambda x: '%9.8f' % x)

In [2]:
## load data, split Xy, upsample with SMOTE, split train/test
data = pd.read_csv('data/RestaurantLessThan20_data_encoded.csv')

## define X and y 
X = data.drop('Y', axis=1).reset_index(drop=True)
y = data.Y

## Handle imbalance
smote = SMOTE()

orig_vc = y.value_counts()
print(f"Orig: {100* orig_vc[1]/(orig_vc.sum()):.2f}% Yes")

X_sm, y_sm = smote.fit_resample(X, y)
sm_vc = y_sm.value_counts()
print(f"After SMOTE: {100* sm_vc[1]/(sm_vc.sum()):.2f}% Yes")

## Data splitting train/test
X_train, X_test, y_train, y_test = train_test_split(X_sm, y_sm, test_size=0.20, random_state=42)

Orig: 70.71% Yes
After SMOTE: 50.00% Yes


In [3]:
X_train.shape

(3152, 55)

## Testing RandomForestClassifier with Grid Search Cross-validation


In [4]:
roc_auc_scorer = make_scorer(roc_auc_score, average='weighted')

In [5]:
cols_to_view = ['rank_test_score', 'param_max_depth', 'param_max_features', 'param_min_samples_split', 
                'param_min_samples_leaf', 'param_n_estimators', 
                'mean_test_score', 'std_test_score', 'mean_train_score', 'std_train_score']

In [7]:
param_grid={'max_depth': [None, 5, 10, 15],
            'max_features': [5, 6, 7, 8, 9, 10],
            'min_samples_split': [2, 3, 4, 5],
            'min_samples_leaf': [1, 3, 5],
            'n_estimators': [100, 150, 200]}

clf = RandomForestClassifier(oob_score=True, random_state=42)

grid_search1 = GridSearchCV(clf, param_grid, scoring=roc_auc_scorer, cv=5, return_train_score=True, n_jobs=3)
grid_search1.fit(X_train.values, y_train.values)

GridSearchCV(cv=5,
             estimator=RandomForestClassifier(oob_score=True, random_state=42),
             n_jobs=3,
             param_grid={'max_depth': [None, 5, 10, 15],
                         'max_features': [5, 6, 7, 8, 9, 10],
                         'min_samples_leaf': [1, 3, 5],
                         'min_samples_split': [2, 3, 4, 5],
                         'n_estimators': [100, 150, 200]},
             return_train_score=True,
             scoring=make_scorer(roc_auc_score, average=weighted))

In [8]:
cv_results1 = pd.DataFrame(grid_search1.cv_results_)
cv_results1.loc[:, cols_to_view].sort_values('rank_test_score').head(10)

,rank_test_score,param_max_depth,param_max_features,param_min_samples_split,param_min_samples_leaf,param_n_estimators,mean_test_score,std_test_score,mean_train_score,std_train_score
79,1,None,7,4,1,150,0.85684356,0.01392176,0.99288743,0.00155211
78,2,None,7,4,1,100,0.85557559,0.01502960,0.99232813,0.00171330
5,3,None,5,3,1,200,0.85557360,0.00885893,0.99794608,0.00057952
651,4,15,5,3,1,100,0.85432281,0.01093255,0.98546954,0.00332819
80,5,None,7,4,1,200,0.85430765,0.01532891,0.99280995,0.00088657
39,6,None,6,3,1,100,0.85366675,0.01463062,0.99746873,0.00126334
4,7,None,5,3,1,150,0.85365168,0.00949922,0.99786746,0.00047311
151,8,None,9,4,1,150,0.85271423,0.01161718,0.99414943,0.00128655
42,9,None,6,4,1,100,0.85242095,0.01317764,0.99114614,0.00085531
40,10,None,6,3,1,150,0.85239075,0.01389711,0.99802583,0.00089788


In [9]:
grid_search1.best_estimator_

RandomForestClassifier(max_features=7, min_samples_split=4, n_estimators=150,
                       oob_score=True, random_state=42)

In [10]:
best_model = grid_search1.best_estimator_
best_model.oob_score_

0.8512055837563451

In [11]:
cross_val_scores = cross_val_score(best_model, X_train, y_train, scoring=roc_auc_scorer, cv=5)
print(f"(weighted avg) AUC: {np.mean(cross_val_scores)} {np.std(cross_val_scores)}")

(weighted avg) AUC: 0.8568435596362913 0.013921755388534886


In [13]:
## fit best model on full train and test on test data
best_model = best_model.fit(X_train, y_train)

## use trained model to get predicted target var
# y_train_pred = best_model.predict(X_train)   
y_test_pred = best_model.predict(X_test) 

# print(f"(weighted avg) AUC: train {roc_auc_score(y_train, y_train_pred, average='weighted')}")
print(f"(weighted avg) AUC: TEST {roc_auc_score(y_test, y_test_pred, average='weighted')}")

(weighted avg) AUC: TEST 0.8554212983261548


# Attempting to look at the resulting decision tree nodes

In [12]:
featimp_df = pd.DataFrame({'feature_name': X.columns, 'importance': best_model.feature_importances_})
featimp_df.sort_values('importance', ascending=False).head(10)

,feature_name,importance
7,time,0.07927435
12,minsToCouponDest,0.06782308
10,income,0.06078961
8,age,0.05592659
16,RestaurantLessThan20,0.05147447
18,destination_No Urgent Place,0.05110185
14,CoffeeHouse,0.04982734
9,education,0.04842292
15,CarryAway,0.04797704
17,Restaurant20To50,0.04555486
